In [ ]:
import os
import moviepy as mp
import speech_recognition as sr

def extrair_audio(video_path, audio_path):
    """
    Extrai o áudio de um arquivo de vídeo usando moviepy e salva como .wav (16kHz, mono, PCM s16le).
    """
    clip = mp.VideoFileClip(video_path)
    clip.audio.write_audiofile(
        audio_path, 
        codec='pcm_s16le', 
        fps=16000, 
        ffmpeg_params=["-ac", "1"]
    )

def reconhecer_fala_por_partes(audio_path, chunk_length=60):
    """
    Transcreve um arquivo de áudio dividindo-o em chunks de 'chunk_length' segundos.
    Retorna a transcrição completa.
    """
    r = sr.Recognizer()
    texto_final = ""

    with sr.AudioFile(audio_path) as source:
        duracao = source.DURATION  # duração total do áudio em segundos
        offset = 0
        
        while offset < duracao:
            # Define quanto tempo de áudio vamos ler neste chunk
            tempo_para_ler = min(chunk_length, duracao - offset)
            
            # Realiza a leitura da fatia
            audio_data = r.record(source, duration=tempo_para_ler)
            
            # Tenta reconhecer este pedaço
            try:
                texto_parcial = r.recognize_google(audio_data, language="pt-BR")
                texto_final += texto_parcial + " "
            except sr.UnknownValueError:
                # Se não conseguiu entender, apenas segue
                pass
            except sr.RequestError as e:
                # Erro de rede ou algo assim
                print(f"Erro na requisição à API: {e}")
                break
            
            # Avança o offset
            offset += chunk_length

    return texto_final.strip()

def processar_videos_em_pasta(pasta):
    """
    Lê todos os arquivos .mp4 na pasta e faz:
      - Extração do áudio em .wav (16kHz, mono)
      - Transcrição (chunk a chunk)
      - Salva o texto em um .txt com o mesmo nome do arquivo de vídeo
    """
    arquivos = os.listdir(pasta)
    caminho_novo_audio = "C:\\Users\\josaf\\Documents\\Aulas\\aulas\\Ciencia-dados\\21-video-para-texto\\audio"
    caminho_novo_texto = "C:\\Users\\josaf\\Documents\\Aulas\\aulas\\Ciencia-dados\\21-video-para-texto\\texto"
    
    for arquivo in arquivos:
        if arquivo.lower().endswith(".mp4"):
            print(f"Processando: {arquivo}")
            
            caminho_video = os.path.join(pasta, arquivo)
            nome_base = os.path.splitext(arquivo)[0]
            caminho_audio = os.path.join(caminho_novo_audio, f"{nome_base}.wav")
            caminho_txt = os.path.join(caminho_novo_texto, f"{nome_base}.txt")
            
            try:
                # Extrair áudio
                extrair_audio(caminho_video, caminho_audio)
                print(f"Áudio extraído em: {caminho_audio}")
                
                # Transcrever em partes
                texto_transcrito = reconhecer_fala_por_partes(caminho_audio, chunk_length=60)
                
                # Salvar .txt
                with open(caminho_txt, "w", encoding="utf-8") as f:
                    f.write(texto_transcrito)

                print(f"Transcrição salva em: {caminho_txt}\n")
            
            except Exception as e:
                print(f"Erro ao processar {arquivo}: {e}")

def main():
    pasta_videos = ""
    if not os.path.isdir(pasta_videos):
        print(f"A pasta '{pasta_videos}' não existe.")
        return
    
    processar_videos_em_pasta(pasta_videos)

if __name__ == "__main__":
    main()
